In [292]:
import pandas as pd
from collections import Counter
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import sys
import copy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.model_selection import cross_val_score, KFold

sys.path.append("../")
import src.data_utils as d_u
import src.feats_generation as f_g

Предварительный анализ данных производится в файле data_exploration.ipynb

Загрузим исходный датасет

In [293]:
df = pd.read_csv("../data/raw/АВСОФТ_тест_ML_приложение.csv")

In [294]:
df.columns

Index(['repository_name', 'commit_hash', 'commit_date', 'commit_author',
       'commit_message', 'bugs'],
      dtype='object')

Удалим столбец, не несущий никакой полезной инфы

In [295]:
df.drop(columns=["commit_hash"], inplace=True)

Преобразуем категориальные признаки в числовые

In [296]:
# OHE для имени репозитория

df = pd.concat([df, pd.get_dummies(df.repository_name)], axis=1)
df.drop(columns=["repository_name"], inplace=True)

In [297]:
# закодируем столбец commit_author, в данном случае я сделаю обычный OHE

df = f_g.ohe(df, "commit_author")  # функция выполняет действия аналогичные ячейке выше

In [298]:
# данные на текущий момент
df.sample(3)

,commit_date,commit_message,bugs,agent,conductor,dockers,mlm,sensor,standard,Alice,Bob,Carol,Dabe,Eve,Mallory,Peggy,Trudy,Victor,Wendy
79,2020-05-20T15:40:58,Импорт одной функции из storage.py вместо *,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,2020-06-12T11:53:10,Поставил lime вместо eli5,3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
191,2020-03-14T11:08:58,suricata pcaplog enabling,3,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0


Сгенерируем признаки из столбца отвечающего за время коммита

In [299]:
# так как изначально тип данного столбца - object
df["commit_date"] = pd.to_datetime(df.commit_date)

df = f_g.encode_work_days(df)
df = f_g.encode_work_hours(df)

In [300]:
df.sample(3)

,commit_date,commit_message,bugs,agent,conductor,dockers,mlm,sensor,standard,Alice,...,Eve,Mallory,Peggy,Trudy,Victor,Wendy,no_work_d,work_d,no_work_h,work_h
35,2020-03-08 17:46:38,Saving unknown labels on emergency saving,9,0,0,0,1,0,0,0,...,0,0,0,0,1,0,1,0,0,1
208,2020-05-14 13:17:47,added session to downloads filename,3,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
309,2020-03-28 11:58:00,added filebeat config,2,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,0,0,1


In [301]:
if "commit_date" in df.columns:
    df.drop(columns=["commit_date"], inplace=True)

Сначала получим бэйзлан без какой-либо информации из сообщения коммита

In [302]:
df_without_msg = df.drop(columns=["commit_message"])

In [303]:
X = df_without_msg.drop(columns=["bugs"])
y = df_without_msg.bugs

In [304]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_idxs, test_idxs = X_train.index, X_test.index

Отмасштабируем значения

In [305]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

Посмотрим качество на разных базовых моделях

In [306]:
zoo_models = [
    LinearRegression(),
    SGDRegressor(random_state=42),
    Ridge(random_state=42),
    Lasso(random_state=42),
    RandomForestRegressor(random_state=42),
    GradientBoostingRegressor(random_state=42),
    SVR(),
    MLPRegressor(random_state=42)
    ]

for model in zoo_models:
    print(f"Модель - {model}")
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(f"Среднеквадратическое отклонение: {mean_squared_error(y_test, preds)}")
    # print(f"Среднее абсолютное отклонение: {mean_absolute_error(y_test, preds)}")
    print()
    

Модель - LinearRegression()
Среднеквадратическое отклонение: 3.599230424388977

Модель - SGDRegressor(random_state=42)
Среднеквадратическое отклонение: 2.792461669579143

Модель - Ridge(random_state=42)
Среднеквадратическое отклонение: 2.813940971574099

Модель - Lasso(random_state=42)
Среднеквадратическое отклонение: 2.5254327662437523

Модель - RandomForestRegressor(random_state=42)
Среднеквадратическое отклонение: 2.6581221678556193

Модель - GradientBoostingRegressor(random_state=42)
Среднеквадратическое отклонение: 2.7976837028211925

Модель - SVR()
Среднеквадратическое отклонение: 2.547624178147599

Модель - MLPRegressor(random_state=42)
Среднеквадратическое отклонение: 2.864029183479513



Посмотрим качество на кросс-валидации для каждой модели

In [307]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)
splits = cv.split(X)

eval_data_lst = []
for i_train, i_test in splits:
    

    X_train, X_test, y_train, y_test = X.loc[i_train], X.loc[i_test], y[i_train], y[i_test]

    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    X_test = ss.transform(X_test)

    eval_data_lst.append((X_train, X_test, y_train, y_test))

for model in zoo_models:
    print(f"Модель - {model}")

    tmp_lst = []

    for X_train, X_test, y_train, y_test in eval_data_lst:
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        tmp_lst.append(mean_squared_error(y_test, preds))
    print(f"Среднеквадратическое отклонение: {np.mean(tmp_lst)}")
    # print(f"Среднее абсолютное отклонение: {mean_absolute_error(y_test, preds)}")
    print() 

Модель - LinearRegression()
Среднеквадратическое отклонение: 3.061867391378663

Модель - SGDRegressor(random_state=42)
Среднеквадратическое отклонение: 2.8427464535834384

Модель - Ridge(random_state=42)
Среднеквадратическое отклонение: 2.8261025740434538

Модель - Lasso(random_state=42)
Среднеквадратическое отклонение: 3.4774931163795175

Модель - RandomForestRegressor(random_state=42)
Среднеквадратическое отклонение: 2.8155012197320235

Модель - GradientBoostingRegressor(random_state=42)
Среднеквадратическое отклонение: 2.8420382249740155

Модель - SVR()
Среднеквадратическое отклонение: 3.0814115021828874

Модель - MLPRegressor(random_state=42)


c:\Users\DungeonMaster3000\.conda\envs\avsoft_test_task\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Среднеквадратическое отклонение: 2.8306670706008417



Теперь посмотрим, как отработают модели, если мы попробуем извлечь информацию из сообщения комита самым простым способом с помощью tfidf

In [308]:
tfidf = TfidfVectorizer(tokenizer=d_u.simple_tokenizer)

msg_embs_train = tfidf.fit_transform(df.commit_message.values[train_idxs]).toarray()
msg_embs_test = tfidf.transform(df.commit_message.values[test_idxs]).toarray()
print(msg_embs_train.shape)
print(msg_embs_test.shape)

(255, 555)
(64, 555)


In [309]:
X = df.drop(columns=["bugs", "commit_message"])
y = df.bugs

msg_embs_train = pd.DataFrame(msg_embs_train)
msg_embs_test = pd.DataFrame(msg_embs_test)

X_train, X_test, y_train, y_test = X.loc[train_idxs], X.loc[test_idxs], y[train_idxs], y[test_idxs]

msg_embs_train.index = train_idxs
msg_embs_test.index = test_idxs

X_train_emb = pd.concat([X_train, msg_embs_train], axis=1)
X_test_emb = pd.concat([X_test, msg_embs_test], axis=1)

ss = StandardScaler()
X_train_emb = ss.fit_transform(X_train_emb.to_numpy())
X_test_emb = ss.transform(X_test_emb.to_numpy())

In [310]:
for model in zoo_models:
    print(f"Модель - {model}")
    model.fit(X_train_emb, y_train)
    preds = model.predict(X_test_emb)
    print(f"Среднеквадратическое отклонение: {mean_squared_error(y_test, preds)}")
    print()

Модель - LinearRegression()
Среднеквадратическое отклонение: 4.321412563493218

Модель - SGDRegressor(random_state=42)
Среднеквадратическое отклонение: 46157.15314686576

Модель - Ridge(random_state=42)
Среднеквадратическое отклонение: 3.4468280846426103

Модель - Lasso(random_state=42)
Среднеквадратическое отклонение: 2.5254327662437523

Модель - RandomForestRegressor(random_state=42)
Среднеквадратическое отклонение: 1.9212890625

Модель - GradientBoostingRegressor(random_state=42)
Среднеквадратическое отклонение: 1.9642904281494462

Модель - SVR()
Среднеквадратическое отклонение: 3.239512981053275

Модель - MLPRegressor(random_state=42)
Среднеквадратическое отклонение: 3.4272161989544205



Можем заметить, что при добавлении информации из сообщения коммита с помощью tfidf мы получили наименьшее отклонение от целевой переменной с помощью моделей RandomForestRegressor и GradientBoostingRegressor.

Для чистоты эксперимента будем создавать tfidf только на основе трэина на каждом разбиении при кросс-валидации

In [311]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)
splits = cv.split(X)

In [312]:
eval_data_lst = []  # список содержащий трэин и тест для каждого разбиения
for i_train, i_test in splits:
    tfidf = TfidfVectorizer(tokenizer=d_u.simple_tokenizer)

    msg_embs_train = tfidf.fit_transform(df.commit_message.values[i_train]).toarray()
    msg_embs_test = tfidf.transform(df.commit_message.values[i_test]).toarray()

    msg_embs_train = pd.DataFrame(msg_embs_train)
    msg_embs_test = pd.DataFrame(msg_embs_test)

    X_train, X_test, y_train, y_test = X.loc[i_train], X.loc[i_test], y[i_train], y[i_test]

    msg_embs_train.index = i_train
    msg_embs_test.index = i_test

    X_train_emb = pd.concat([X_train, msg_embs_train], axis=1)
    X_test_emb = pd.concat([X_test, msg_embs_test], axis=1)

    ss = StandardScaler()
    X_train_emb = ss.fit_transform(X_train_emb.to_numpy())
    X_test_emb = ss.transform(X_test_emb.to_numpy())

    # X_train_emb = X_train_emb.to_numpy()
    # X_test_emb = X_test_emb.to_numpy()

    eval_data_lst.append((X_train_emb, X_test_emb, y_train, y_test))    

In [313]:
for model in zoo_models:
    print(f"Модель - {model}")

    tmp_lst = []

    for X_train_emb, X_test_emb, y_train, y_test in eval_data_lst:
        model.fit(X_train_emb, y_train)
        preds = model.predict(X_test_emb)
        tmp_lst.append(mean_squared_error(y_test, preds))
    print(f"Среднеквадратическое отклонение: {np.mean(tmp_lst)}")
    # print(f"Среднее абсолютное отклонение: {mean_absolute_error(y_test, preds)}")
    print("________________________________________________")

Модель - LinearRegression()
Среднеквадратическое отклонение: 3.840936851252214
________________________________________________
Модель - SGDRegressor(random_state=42)
Среднеквадратическое отклонение: 66171.03174002295
________________________________________________
Модель - Ridge(random_state=42)
Среднеквадратическое отклонение: 3.5483044505545527
________________________________________________
Модель - Lasso(random_state=42)
Среднеквадратическое отклонение: 3.4774931163795175
________________________________________________
Модель - RandomForestRegressor(random_state=42)
Среднеквадратическое отклонение: 2.55626306547619
________________________________________________
Модель - GradientBoostingRegressor(random_state=42)
Среднеквадратическое отклонение: 2.3125281732052922
________________________________________________
Модель - SVR()
Среднеквадратическое отклонение: 3.92950610268129
________________________________________________
Модель - MLPRegressor(random_state=42)
Среднеквадрати

Лучшее качество получается на моделях RandomForestRegressor и GradientBoostingRegressor. Попробуем то же самое, но без использования StandardScaler

In [314]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)
splits = cv.split(X)

eval_data_lst = []  # список содержащий трэин и тест для каждого разбиения
for i_train, i_test in splits:
    tfidf = TfidfVectorizer(tokenizer=d_u.simple_tokenizer)

    msg_embs_train = tfidf.fit_transform(df.commit_message.values[i_train]).toarray()
    msg_embs_test = tfidf.transform(df.commit_message.values[i_test]).toarray()

    msg_embs_train = pd.DataFrame(msg_embs_train)
    msg_embs_test = pd.DataFrame(msg_embs_test)

    X_train, X_test, y_train, y_test = X.loc[i_train], X.loc[i_test], y[i_train], y[i_test]

    msg_embs_train.index = i_train
    msg_embs_test.index = i_test

    X_train_emb = pd.concat([X_train, msg_embs_train], axis=1)
    X_test_emb = pd.concat([X_test, msg_embs_test], axis=1)

    X_train_emb = X_train_emb.to_numpy()
    X_test_emb = X_test_emb.to_numpy()

    eval_data_lst.append((X_train_emb, X_test_emb, y_train, y_test))   

for model in zoo_models:
    print(f"Модель - {model}")

    tmp_lst = []

    for X_train_emb, X_test_emb, y_train, y_test in eval_data_lst:
        model.fit(X_train_emb, y_train)
        preds = model.predict(X_test_emb)
        tmp_lst.append(mean_squared_error(y_test, preds))
    print(f"Среднеквадратическое отклонение: {np.mean(tmp_lst)}")
    # print(f"Среднее абсолютное отклонение: {mean_absolute_error(y_test, preds)}")
    print("________________________________________________")

Модель - LinearRegression()
Среднеквадратическое отклонение: 4.051258697962091
________________________________________________
Модель - SGDRegressor(random_state=42)
Среднеквадратическое отклонение: 2.544314136103807
________________________________________________
Модель - Ridge(random_state=42)
Среднеквадратическое отклонение: 2.569979525885581
________________________________________________
Модель - Lasso(random_state=42)
Среднеквадратическое отклонение: 3.4774931163795175
________________________________________________
Модель - RandomForestRegressor(random_state=42)
Среднеквадратическое отклонение: 2.55626306547619
________________________________________________
Модель - GradientBoostingRegressor(random_state=42)
Среднеквадратическое отклонение: 2.3125281732052922
________________________________________________
Модель - SVR()
Среднеквадратическое отклонение: 2.75252931290019
________________________________________________
Модель - MLPRegressor(random_state=42)


c:\Users\DungeonMaster3000\.conda\envs\avsoft_test_task\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\DungeonMaster3000\.conda\envs\avsoft_test_task\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\DungeonMaster3000\.conda\envs\avsoft_test_task\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\DungeonMaster3000\.conda\envs\avsoft_test_task\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the 

Среднеквадратическое отклонение: 2.7325733258057694
________________________________________________


c:\Users\DungeonMaster3000\.conda\envs\avsoft_test_task\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


На кросс-валидации также видно, что при использование tfidf уменьшается среднее квадратичное отклоанение, причем если не проводить масштабирование, то в лидеры помимо моделей RandomForestRegressor и GradientBoostingRegreessor попадают и модели SGDRegressor и Ridge. Также имеет смысл поэксперементировать с архитектурой MLPRegressor и увеличением числа итераций.

На данный момент лучшее значение на mse - 2.3125281732052922, которое было получено с помощь GradientBoostingRegressor